In [2]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [3]:
from keras.datasets import mnist
data = mnist.load_data()

11493376/11490434 [==============================] - 37s 3us/step


In [36]:
(X_train,y_train),(X_test,y_test) = data

##  2.1网络架构 

In [37]:
from keras import models
from keras import layers
import tensorflow as tf
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

神经网络的核心组件是层（layer），它是一种数据处理模块,可以将它看成数据过滤器。进去一些数据，出来的数据变得更加有用.具体来说，层从输入数据中提取表示——我们期望这种表示有助于解决手头的问题。大多数深度学习都是将简单的层链接起来，从而实现渐进式的数据蒸馏（data distillation）。深度学习模型就像是数据处理的筛子，包含一系列越来越精细的数据过滤器（即层）。
本例中的网络包含2个Dense层，它们是密集连接（也叫全连接）的神经层。第二层（也是最后一层）是一个 10 路 softmax 层，它将返回一个由 10 个概率值（总和为 1）组成的数组。每个概率值表示当前数字图像属于 10 个数字类别中某一个的概率。要想训练网络，我们还需要选择编译（compile）步骤的三个参数

- 损失函数 ： 网络如何衡量在训练数据上的性能，即网络如何朝着正确的方向前进。
- 优化器 ：基于训练数据和损失函数来更新网络的机制。
- 在训练和测试过程中需要监控的指标（metric）衡量网络/算法的好处

## 2.2编译

In [38]:
network.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

## 2.3 准备数据，数据归一化处理
 - 数据变化到0-1之间
 - 单个数据矩阵变向量

In [39]:
X_train = X_train.reshape((60000, 28 * 28))
X_train = X_train.astype('float32') / 255
X_test = X_test.reshape((10000, 28 * 28))
X_test = X_test.astype('float32') / 255

In [40]:
y_train.shape

(60000,)

In [41]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

(60000, 10)

In [42]:
network.fit(X_train,y_train, epochs=5, batch_size=128)

W0828 13:49:09.445996 140240796337984 deprecation.py:323] From /home/admin/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0828 13:49:09.522215 140240796337984 deprecation_wrapper.py:119] From /home/admin/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/5
60000/60000 [==============================] - 28s 459us/step - loss: 0.2568 - acc: 0.9259
Epoch 2/5
60000/60000 [==============================] - 27s 445us/step - loss: 0.1034 - acc: 0.9695
Epoch 3/5
60000/60000 [==============================] - 26s 435us/step - loss: 0.0678 - acc: 0.9796
Epoch 4/5
60000/60000 [==============================] - 26s 436us/step - loss: 0.0491 - acc: 0.9856
Epoch 5/5
60000/60000 [==============================] - 27s 446us/step - loss: 0.0369 - acc: 0.9887


In [43]:
 test_loss, test_acc = network.evaluate(X_test, y_test)

10000/10000 [==============================] - 7s 735us/step


In [44]:
test_acc  # 测试数据的准确率

0.9785

In [45]:
test_loss # 损失

0.07209754027300515

#### ---------------------------------------------------------- 第一个网络训练结束----------------------------------------------------------

# 第二章：神经网络的数学基础

<a name="zDk8K"></a>
## 2.0 本章要点
本章所掌握的技能：

- 神经网络的数据表示
- 常见数据的理解与认识


神经网络的核心组件是层（layer），它是一种数据处理模块,可以将它看成数据过滤器。进去一些数据，出来的数据变得更加有用.具体来说，层从输入数据中提取表示——我们期望这种表示有助于解决手头的问题。大多数深度学习都是将简单的层链接起来，从而实现渐进式的数据蒸馏（data distillation）。深度学习模型就像是数据处理的筛子，包含一系列越来越精细的数据过滤器（即层）。<br />本例中的网络包含2个Dense层，它们是密集连接（也叫**全连接**）的神经层。第二层（也是最后一层）是一个 10 路 softmax 层，它将返回一个由 10 个概率值（总和为 1）组成的数组。每个概率值表示当前数字图像属于 10 个数字类别中某一个的概率。要想训练网络，我们还需要选择编译（compile）步骤的三个参数。
<a name="bbdtY"></a>
## 2.1 神经网络的数据表示
一般来说，当前所有机器学习系统都使用张量作为基本数据结构。张量对这个领域非常重要，重要到 Google 的<br />TensorFlow 都以它来命名。那么什么是张量？张量这一概念的核心在于，它是一个数据容器。它包含的数据几乎总是数值数据，因此它是数字的容器。你可能对矩阵很熟悉，它是二维张量。张量是矩阵向任意维度的推广［注意，张量的维度（dimension）通常叫作轴（axis）］
<a name="W5DDA"></a>
### 2.1.1 标量(0D张量)

- 仅包含一个数字的张量叫作标量（scalar，也叫标量张量、零维张量、0D 张量）。在 Numpy中，一个 float32 或 float64 的数字就是一个标量张量（或标量数组）。即一个数字
<a name="T0Luh"></a>
### 2.1.2 向量(1D 张量)

- 数字组成的数组叫作向量（vector）或一维张量（1D 张量）。一维张量只有一个轴

<a name="YZJqP"></a>
### 2.1.3 矩阵（2D 张量）

- 向量组成的数组叫作矩阵（matrix）或二维张量（2D 张量）。矩阵有 2 个轴（通常叫作行和列）。你可以将矩阵直观地理解为数字组成的矩形网格
<a name="0drtq"></a>
### 2.1.4 （3D张量与更高维张量）

- 将多个矩阵组合成一个新的数组，可以得到一个 3D 张量
<a name="TVM3P"></a>
### 2.1.5 数据批量的概念  

- 通常来说，深度学习中所有数据张量的第一个轴（0 轴，因为索引从 0 开始）都是样本轴（samples axis，有时也叫样本维度）。在 MNIST 的例子中，样本就是数字图像。此外，深度学习模型不会同时处理整个数据集，而是将数据拆分成小批量。
<a name="CnLIV"></a>
## 2.2 数据的理解与认识
<a name="zm73D"></a>
### 2.2.1 时间序列数据/序列数据 

- 股票价格数据集。每一分钟，我们将股票的当前价格、前一分钟的最高价格和前一分钟的最低价格保存下来。因此每分钟被编码为一个3D向量，整个交易日被编码为一个形
状为 (390, 3) 的 2D 张量（一个交易日有 390 分钟），而 250 天的数据则可以保存在一
个形状为 (250, 390, 3) 的 3D 张量中。这里每个样本是一天的股票数据。
- 推文数据集。我们将每条推文编码为 280 个字符组成的序列，而每个字符又来自于 128
个字符组成的字母表。在这种情况下，每个字符可以被编码为大小为 128 的二进制向量
（只有在该字符对应的索引位置取值为 1，其他元素都为 0）。那么每条推文可以被编码
为一个形状为 (280, 128) 的 2D 张量，而包含 100 万条推文的数据集则可以存储在一
个形状为 (1000000, 280, 128) 的张量中。

<a name="SKjkL"></a>
### 2.2.2 图片数据 
图像通常具有三个维度：**高度、宽度和颜色深度**。虽然灰度图像（比如 MNIST 数字图像）
只有一个颜色通道，因此可以保存在 2D 张量中，但按照惯例，图像张量始终都是 3D 张量，灰
度图像的彩色通道只有一维。因此，如果图像大小为 256×256，那么 128 张灰度图像组成的批
量可以保存在一个形状为 (128, 256, 256, 1) 的张量中，而 128 张彩色图像组成的批量则可以保存在一个形状为 (128, 256, 256, 3) 的张量中。<br />图像张量的形状有两种约定：通道在后（channels-last）的约定（在 TensorFlow 中使用）和
通道在前（channels-first）的约定（在 Theano 中使用）。Google 的 TensorFlow 机器学习框架将
颜色深度轴放在最后：(samples, height, width, color_depth)。与此相反，Theano
将图像深度轴放在批量轴之后：(samples, color_depth, height, width)。如果采
用 Theano 约定，前面的两个例子将变成 (128, 1, 256, 256) 和 (128, 3, 256, 256)。
Keras 框架同时支持这两种格式。<br />
<br />![01.png](https://cdn.nlark.com/yuque/0/2019/png/280095/1566977731196-293bbd3e-678f-47aa-ac6e-3967687584e5.png#align=left&display=inline&height=137&name=01.png&originHeight=323&originWidth=630&size=15383&status=done&width=267)

<a name="Gz04z"></a>
### 2.2.3 视频数据 
视频数据是现实生活中需要用到 5D 张量的少数数据类型之一。视频可以看作一系列帧，
每一帧都是一张彩色图像。由于每一帧都可以保存在一个形状为 (height, width, color_
depth) 的 3D 张量中，因此一系列帧可以保存在一个形状为 (frames, height, width,
color_depth) 的 4D 张量中，而不同视频组成的批量则可以保存在一个 5D 张量中，其形状为
(samples, frames, height, width, color_depth)。
举个例子，一个以每秒 4 帧采样的 60 秒 YouTube 视频片段，视频尺寸为 144×256，这个
视频共有 240 帧。4 个这样的视频片段组成的批量将保存在形状为 (4, 240, 144, 256, 3)
的张量中。总共有 106 168 320 个值！如果张量的数据类型（dtype）是 float32，每个值都是
32 位，那么这个张量共有 405MB。在现实生活中遇到的视频要小得多，因为它们不以
float32 格式存储，而且通常被大大压缩，比如 MPEG 格式。













